# Importing All Excels (with ret flag=1)

In [1]:
import os
import pandas as pd
import numpy as np
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Border, Side, Font
from openpyxl.utils import get_column_letter
from openpyxl.worksheet.table import Table, TableStyleInfo
from openpyxl.styles.alignment import Alignment

In [2]:
#########Add file paths here for configuration
cwd = os.getcwd()

#combined_output_file_name = os.path.join(cwd, "final_outputs", '09-05-2024 Full Data_v0.9.xlsx')
combined_output_file_name = '09-05-2024 Full Data_v0.9.xlsx'


one_time_prep = os.path.join(cwd, "client_data", "One_time_prep.xlsx")
rm_transfer = os.path.join(cwd, "client_data", "RM_Transfer_Costs.xlsx")
calendar_factory_operations = os.path.join(cwd,"client_data","Calendar_Factory_Operations.xlsx")
demand_output = "28-04-2024_Demand_Automation.xlsx"
capacity_output = "Capacity_DD_Filtered.xlsx"
production_output = "03-05-2024_Prod_Automation.xlsx"
bom_output="BOM Top 80 Output.xlsx"
rm_supply_output = "RM Supply.xlsx"
labour_output = '22-04-2024 labour Data.xlsx'
fg_sku_list = '29_04_2024_FG-SKU.xlsx'
rm_sku_list = '03-05-2024 RMSKU Master List.xlsx'
primary_freight = '03-05-2024 Primary Freight.xlsx'
secondary_freight = '03-05-2024 Secondary Freight.xlsx'
wh_fg_storage = '29-04-2024 WH-FG storage.xlsx'
trimming_output = "29-04-24_Trimming_Output.csv"
csnorm_output = "09-05-24_FG MaxCS Output.xlsx"
fg_m1_os = '29-04-24_FG M10S Output.xlsx'
rm_m1_os = "03-05-2024 RM-M1OS.xlsx"

In [3]:
df_production=pd.read_excel(production_output,sheet_name='Production_output',skiprows=1,usecols=lambda column: column != 'Unnamed: 0')
df_BOM=pd.read_excel(bom_output,skiprows=1,usecols=lambda column: column != 'Unnamed: 0')
df_Factory_Operations = pd.read_excel(calendar_factory_operations, skiprows=1, sheet_name='Factory Operations', usecols=lambda column: column != 'Unnamed: 0')
df_Calendar = pd.read_excel(calendar_factory_operations, skiprows=1, sheet_name='Calendar', usecols=lambda column: column != 'Unnamed: 0')
df_RM_Supply=pd.read_excel(rm_supply_output,sheet_name='RM Supply',skiprows=1,usecols=lambda column: column != 'Unnamed: 0')
df_LabourData=pd.read_excel(labour_output,skiprows=1,sheet_name='Labour Data',usecols=lambda column: column != 'Unnamed: 0')
df_rmtransfer=pd.read_excel(rm_transfer,sheet_name='RM Transfer', skiprows=3, usecols=lambda column: column != 'Unnamed: 0')
df_FGSKU=pd.read_excel(fg_sku_list,sheet_name='FG SKU',skiprows=1,usecols=lambda column: column != 'Unnamed: 0')

df_RMSKU = pd.read_excel(rm_sku_list, skiprows=1, sheet_name='RMSKU Master', usecols=lambda column: column != 'Unnamed: 0')
df_RMSKU = df_RMSKU.drop(columns=['FactCode']).drop_duplicates()
df_pri_frt=pd.read_excel(primary_freight,sheet_name='Primary Freight',skiprows=1,usecols=lambda column: column != 'Unnamed: 0')
df_sec_frt=pd.read_excel(secondary_freight,sheet_name='Secondary Freight',skiprows=1,usecols=lambda column: column != 'Unnamed: 0')
df_demand=pd.read_excel(demand_output)
df_whfg=pd.read_excel(wh_fg_storage,skiprows=1,usecols=lambda column: column != 'Unnamed: 0')
df_trim = pd.read_csv(trimming_output, usecols=lambda column: column != 'Unnamed: 0')

df_FG_M1OS=pd.read_excel(fg_m1_os,skiprows=1,sheet_name='FG-M1OS', usecols=lambda column: column != 'Unnamed: 0')
df_RM_M1OS=pd.read_excel(rm_m1_os,skiprows=1, usecols=lambda column: column != 'Unnamed: 0')


TC_04_INV_06_FGCSInvNorm = pd.read_excel(csnorm_output,skiprows=1,usecols=lambda column: column != 'Unnamed: 0')
TC_04_INV_06_FGCSInvNorm=TC_04_INV_06_FGCSInvNorm[TC_04_INV_06_FGCSInvNorm['retain_flag']==1]
TC_04_INV_06_FGCSInvNorm.drop(columns={'retain_flag'}, inplace=True)


In [4]:
####FROM ONE TIME PREP
df_country = pd.read_excel(one_time_prep, skiprows=4, sheet_name='Country', usecols=lambda column: column != 'Unnamed: 0')
df_warehouse = pd.read_excel(one_time_prep, skiprows=4, sheet_name='Warehouse', usecols=lambda column: column != 'Unnamed: 0')
df_factory=pd.read_excel(one_time_prep,skiprows=4,sheet_name='Factory', usecols=lambda column: column != 'Unnamed: 0')
df_lines=pd.read_excel(one_time_prep,skiprows=4,sheet_name='Lines', usecols=lambda column: column != 'Unnamed: 0')
df_trimming_SKU=pd.read_excel(one_time_prep,skiprows=4,sheet_name='TrimmingSKUs', usecols=lambda column: column != 'Unnamed: 0')

#Preparation Line Capacity Cons
df_prep_line_cap1 = pd.read_excel(one_time_prep, skiprows=4, sheet_name='Preparation Lines FG', usecols=lambda column: column != 'Unnamed: 0')
df_prep_line_cap1=df_prep_line_cap1[['Factory','Combination','SKU Code']].drop_duplicates()

df_prep_line_cap2 = pd.read_excel(one_time_prep, skiprows=4, sheet_name='Preparation Lines Capacity', usecols=lambda column: column != 'Unnamed: 0')
df_prep_line_cap2=df_prep_line_cap2[['Factory','Combination','maxProdKg']].drop_duplicates()
                                  
#RM Transfer Restrictions
df_RM_transfer_res= pd.read_excel(one_time_prep, skiprows=4, sheet_name='RM Transfer Restrictions', usecols=lambda column: column != 'Unnamed: 0')

#Minimum Contract Labour Hours
df_Min_contract_labour_hours= pd.read_excel(one_time_prep, skiprows=4, sheet_name='Minimum Contract Labour Hours', usecols=lambda column: column != 'Unnamed: 0')

In [5]:
#Production
##Preprocessing all datasets


# FG - M1OS
df_FG_M1OS= df_FG_M1OS[df_FG_M1OS['retain_flag']==1]
df_FG_M1OS = df_FG_M1OS.drop(columns=['FACTCode']).drop_duplicates()

# RM-M1OS
df_RM_M1OS= df_RM_M1OS[df_RM_M1OS['Retain_Flag']==1]

df_production= df_production[df_production['retain_flag']==1]
df_production['MOQ']=0
df_production['EnergyCostPerUnitBOM1']=df_production['EnergyCostPerUnit']

df_production['EnergyCostPerUnitBOM2']=df_production['EnergyCostPerUnit']               

df_production['TeamSizeBOM1']=df_production['CrewSize']               

df_production['TeamSizeBOM2']=df_production['CrewSize']

df_production.drop(columns=['EnergyCostPerUnit','CrewSize'], inplace=True) 

new_column_order = ['FactCode', 'LineNum', 'Unified code', 'MOQ', 'ThruputPerHour', 'EnergyCostPerUnitBOM1',
                    'EnergyCostPerUnitBOM2', 'TeamSizeBOM1', 'TeamSizeBOM2', 'retain_flag']

# Reorder the columns
df_production = df_production.reindex(columns=new_column_order)


# BOM
df_BOM= df_BOM[df_BOM['retain_flag']==1]
df_BOM.drop(columns={'Total Packaging Cost','Total Cost','Cumulative % Share'}, inplace= True)


#Preparation Line Capacity Cons
df_prep_line_cap1=df_prep_line_cap1[['Factory','Combination','SKU Code']].drop_duplicates()

df_prep_line_cap2=df_prep_line_cap2[['Factory','Combination','maxProdKg']].drop_duplicates()

# RM Supply
df_RM_Supply= df_RM_Supply[df_RM_Supply['retain_flag']==1]


# LabourData
df_LabourData.drop(columns={'C_Min','C_Max','C_Hrs','C_Wages'}, inplace=True)

#RM Transfer Restrictions

#Minimum Contract Labour Hours

#RM Transfer
df_rmtransfer.drop(columns={'Freight Cost/Kg','Packaging Cost/Kg'}, inplace= True)

#FGSKU
df_FGSKU= df_FGSKU[df_FGSKU['retain_flag']==1]
df_FGSKU.drop(columns={'UOMPerPallet','TrimKGPerUnit'}, inplace= True)

#Primary Freight
df_pri_frt= df_pri_frt[df_pri_frt['retain_flag']==1]
df_pri_frt['Total Cost(FRT_CD)'] = df_pri_frt['Total Cost(FRT_CD)'].fillna(0)

#Secondary frt
df_sec_frt= df_sec_frt[df_sec_frt['retain_flag']==1]
df_sec_frt['Total Cost(FRT_CD)'] = df_sec_frt['Total Cost(FRT_CD)'].fillna(0)
#df_sec_frt.drop(columns={'UOM_x'}, inplace=True)

# Demand
df_demand= df_demand[df_demand['Ret_Flag']==1]

# WH-FG Storage
df_whfg= df_whfg[df_whfg['retain_flag']==1]
df_whfg.rename(columns={'Storage Cost/Carton': 'Storage Cost per UOM'}, inplace=True)


# Mapping dictionary for grouping
group_mapping = {
    'NF1': 'NFC',
    'NF2': 'NFC',
    'KPF': 'KFC',
    'GFM': 'GFC'
}

# Replace Comp Org values with grouped names
df_trim['COMP_ORG'] = df_trim['COMP_ORG'].replace(group_mapping)

# Calculate mean for items present in both NF1 and NF2
df_trim_grouped = df_trim.groupby(['ASSEMB_ITEM_CODE', 'COMP_ORG']).agg({'Trimming(in KG)': 'mean'}).reset_index()

# Rename columns
df_trim_grouped.columns = ['ASSEMB_ITEM_CODE', 'Fact Code', 'TrimmingKGPerUnit']


df_FGSKU1 = pd.DataFrame({'FG Code': df_FGSKU['FG Code'].unique()})

# Merge FG SKU with trimmed data
df_merge_fg_trim = pd.merge(df_FGSKU1, df_trim_grouped, left_on='FG Code', right_on='ASSEMB_ITEM_CODE', how='inner')
df_merge_fg_trim=df_merge_fg_trim.drop(columns='ASSEMB_ITEM_CODE')


###### New part adding here:#############
df_pri_frt['MaxShipment'] = 1000000.0

df_sec_frt['MaxShipment'] = 1000000.0


df_RM_Supply['MinCS']=0.0
df_RM_Supply['MaxCS']=1000000.0
df_RM_Supply['MaxSupLim']=1.0
df_RM_Supply['RMStoreCost']=1.0


df_Factory_Operations['MaxVirtualShift']=1000.0
df_Factory_Operations['CostVirtualShift']=1000.0
df_Factory_Operations['CostRealShift']=100.0
df_Factory_Operations['CostOT']=20.0

df_LabourData['RegCSMin']=0.0
df_LabourData['RegCSMax']=1000.0
df_LabourData['CtrCSMin']=0.0
df_LabourData['CtrCSMax']=1000.0
df_LabourData['RegMaxHire'] = np.where(df_LabourData['MonthNum'] <= 3, 0.0, 1000.0)
df_LabourData['RegMaxFire'] = np.where(df_LabourData['MonthNum'] <= 3, 0.0, 1000.0)
df_LabourData['CtrMaxHire'] = np.where(df_LabourData['MonthNum'] <= 3, 0.0, 1000.0)
df_LabourData['CtrMaxFire'] = np.where(df_LabourData['MonthNum'] <= 3, 0.0, 1000.0)


# Define columns and data types for TC_04_INV_03_FGInv
columns_2 = {
    'WHCode': 'object',
    'FGSKUCode': 'object',
    'MthNum': 'int',
    'MaxOS': 'float',
    'MinOS': 'float',
    'MaxOSDays': 'float',
    'MinOSDays': 'float',
    'MinCS': 'float',
    'MaxCS': 'float',
    'MaxSumLim': 'float'
}

# Create TC_04_INV_03_FGInv DataFrame
TC_04_INV_03_FGInv = pd.DataFrame(columns=columns_2)
TC_04_INV_03_FGInv['MinCS']=0.0
TC_04_INV_03_FGInv['MaxCS']=1000000.0
TC_04_INV_03_FGInv['MaxSupLim']=1.0

#######

data = {
    'CountryCode': [None, None, None],
    'FactCode': [None, None, None],
    'FGSKUCode': [None, None, None],
    'SupplyStatus': [None, None, None]
}

TC_06_NegativeList_01_FGSupply = pd.DataFrame(data)

# Converting columns to nullable data types
TC_06_NegativeList_01_FGSupply = TC_06_NegativeList_01_FGSupply.astype({
    'CountryCode': 'string',
    'FactCode': 'string',
    'FGSKUCode': 'string',
    'SupplyStatus': 'string'
})

########## TC_06_NegativeList_02_RMSupply ##########

data1 = {
    'FactCode': [None, None, None],
    'RMSKUCode': [None, None, None],
    'MthNum': [None, None, None],
    'SupplyStatus': [None, None, None]
}

TC_06_NegativeList_02_RMSupply = pd.DataFrame(data1)


TC_06_NegativeList_02_RMSupply = TC_06_NegativeList_02_RMSupply.astype({
    'FactCode': 'string',
    'RMSKUCode': 'string',
    'MthNum': 'Int64',
    'SupplyStatus': 'string'
})

df_factory['TrimStorageCost'] = None  # Initialize the column with None values


# Convert the column to float data type
df_factory['TrimStorageCost'] = df_factory['TrimStorageCost'].astype('float')
#########
# df_demand['WH'] = None
# df_demand['PLT'] = None
# df_demand['Line'] = None
# df_demand['TrimSKU'] = None
# df_demand['RMSKU'] = None
# df_demand['M09'] = None
# df_demand['M10'] = None

# df_demand = df_demand.astype({
#     'WH': 'string',
#     'PLT': 'string',
#     'Line': 'string',
#     'TrimSKU': 'string',
#     'RMSKU': 'string',
#     'M09': 'string',
#     'M10': 'string'
# })

#########
# df_sec_frt['PLT'] = None
# df_sec_frt['LINE'] = None
# df_sec_frt['TrimSKU'] = None
# df_sec_frt['RMSKU'] = None
# df_sec_frt['MthNum'] = None
# df_sec_frt['M09'] = None
# df_sec_frt['M10'] = None
# df_sec_frt['Max_GFC'] = None
# df_sec_frt['Max_KFC'] = None
# df_sec_frt['Max_NFC'] = None
# df_sec_frt['FGWtPerUnit'] = None

# Converting columns to the specified data types
# df_sec_frt = df_sec_frt.astype({
#     'PLT': 'string',
#     'LINE': 'string',
#     'TrimSKU': 'string',
#     'RMSKU': 'string',
#     'MthNum': 'Int64',
#     'M09': 'string',
#     'M10': 'string',
#     'Max_GFC': 'float',
#     'Max_KFC': 'float',
#     'Max_NFC': 'float',
#     'FGWtPerUnit': 'float'
# })
# ###############
# df_pri_frt['CTY'] = None
# df_pri_frt['LINE'] = None
# df_pri_frt['TrimSKU'] = None
# df_pri_frt['RMSKU'] = None
# df_pri_frt['MthNum'] = None
# df_pri_frt['M09'] = None
# df_pri_frt['M10'] = None
# df_pri_frt['MaxQty'] = None
# df_pri_frt['FGWtPerUnit'] = None

# Converting columns to the specified data types
# df_pri_frt = df_pri_frt.astype({
#     'CTY': 'string',
#     'LINE': 'string',
#     'TrimSKU': 'string',
#     'RMSKU': 'string',
#     'MthNum': 'Int64',
#     'M09': 'string',
#     'M10': 'string',
#     'MaxQty': 'float',
#     'FGWtPerUnit': 'float'
# })
##########################

df_LabourData['RegCSMin'] = None
df_LabourData['RegCSMax'] = None
df_LabourData['CtrCSMin'] = None
df_LabourData['CtrCSMax'] = None
df_LabourData['RegMaxHire'] = None
df_LabourData['RegMaxFire'] = None
df_LabourData['CtrMaxHire'] = None
df_LabourData['CtrMaxFire'] = None

# Converting columns to the specified data types
df_LabourData = df_LabourData.astype({
    'RegCSMin': 'float',
    'RegCSMax': 'float',
    'CtrCSMin': 'float',
    'CtrCSMax': 'float',
    'RegMaxHire': 'float',
    'RegMaxFire': 'float',
    'CtrMaxHire': 'float',
    'CtrMaxFire': 'float'
})

###################
df_RM_Supply['MinCS'] = None
df_RM_Supply['MaxCS'] = None
df_RM_Supply['MaxSupLim'] = None
df_RM_Supply['RMStoreCost'] = None

# Converting columns to the specified data types
df_RM_Supply = df_RM_Supply.astype({
    'MinCS': 'float',
    'MaxCS': 'float',
    'MaxSupLim': 'float',
    'RMStoreCost': 'float'
})
###############################
df_lines['LineDesc']= None

df_lines['LineDesc']=df_lines['LineDesc'].astype('string')


data2 = {'ICC': [0]}  # Provide a list with a single None value
TM_04_FIN_01_ICC = pd.DataFrame(data2)

TM_04_FIN_01_ICC['ICC'] = TM_04_FIN_01_ICC['ICC'].astype('float')

data3 = {
    'MthNum': [None],
    'MthDesc': [None]
}

TM_03_CAL_01_Months = pd.DataFrame(data3)

# Assuming 'MthNum' is supposed to be an integer column and 'MthDesc' a string column
TM_03_CAL_01_Months['MthNum'] = TM_03_CAL_01_Months['MthNum'].astype('Int64')
TM_03_CAL_01_Months['MthDesc'] = TM_03_CAL_01_Months['MthDesc'].astype('string')

df_country['SubCountryCode']= None
df_country['SubCountryDesc']= None
df_country['SubCountryCode'] = df_country['SubCountryCode'].astype('string')

df_country['SubCountryDesc'] = df_country['SubCountryDesc'].astype('string')

# Create a dictionary with DataFrame names as keys and DataFrames as values
dfs = {
    'df_country': df_country,
    'df_warehouse': df_warehouse,
    'df_factory': df_factory,
    'df_lines': df_lines,
    'df_RMSKU': df_RMSKU,
    'df_trimming_SKU': df_trimming_SKU,
    'df_FG_M1OS': df_FG_M1OS,
    'df_RM_M1OS': df_RM_M1OS,
    'df_production': df_production,
    'df_pri_frt' : df_pri_frt,
    'df_sec_frt': df_sec_frt,
    'df_BOM': df_BOM,
    'df_Factory_Operations': df_Factory_Operations,
    'df_Calendar': df_Calendar,
    'df_RM_Supply': df_RM_Supply,
    'df_LabourData': df_LabourData,
    'df_RM_transfer_res': df_RM_transfer_res,
    'df_Min_contract_labour_hours': df_Min_contract_labour_hours,
    'df_rmtransfer': df_rmtransfer,
    'df_demand': df_demand,
    'df_FGSKU' : df_FGSKU,
    'df_whfg' : df_whfg,
    'df_prep_line_cap2': df_prep_line_cap2,
    'df_prep_line_cap1' : df_prep_line_cap1,
    'df_merge_fg_trim' : df_merge_fg_trim,
    'TC_04_INV_06_FGCSInvNorm' : TC_04_INV_06_FGCSInvNorm,
    'TC_04_INV_03_FGInv': TC_04_INV_03_FGInv,
    'TC_06_NegativeList_01_FGSupply':TC_06_NegativeList_01_FGSupply,
    'TC_06_NegativeList_02_RMSupply':TC_06_NegativeList_02_RMSupply,
    'TM_04_FIN_01_ICC': TM_04_FIN_01_ICC,
    'TM_03_CAL_01_Months': TM_03_CAL_01_Months
}

# Iterate over the dictionary and print each DataFrame
for df_name, df in dfs.items():
    print(f"DataFrame: {df_name}")
    print(df)
    print("\n")


DataFrame: df_country
              CountryCode           CountryDesc SubCountryCode SubCountryDesc
0   IS & PL_Cote d'Ivoire         Cote d'Ivoire           <NA>           <NA>
1            IS & PL_Iraq                  Iraq           <NA>           <NA>
2          IS & PL_Jordan                Jordan           <NA>           <NA>
3             IS & PL_KSA          Saudi Arabia           <NA>           <NA>
4             IS & PL_UAE  United Arab Emirates           <NA>           <NA>
5                 KSA_KSA          Saudi Arabia           <NA>           <NA>
6           Kuwait_Kuwait                Kuwait           <NA>           <NA>
7             QSR_Bahrain               Bahrain           <NA>           <NA>
8                QSR_Iraq                  Iraq           <NA>           <NA>
9              QSR_Jordan                Jordan           <NA>           <NA>
10            QSR_Jordan                 Jordan           <NA>           <NA>
11                QSR_KSA          Saudi A

# Formatting into single excel

In [6]:
#main Block

# Write the name of dataframe and tab name that you want in excel
dataframes = {
    
'Country': df_country,
'Warehouse': df_warehouse,
'Factory': df_factory,
'Lines': df_lines,
'FG SKUs': df_FGSKU,
'FG- Fact Trimming' : df_merge_fg_trim,
'RM SKUs': df_RMSKU,
'TrimmingSKUs': df_trimming_SKU,
'FG - M1OS': df_FG_M1OS,
'RM- M1OS': df_RM_M1OS,
'Production': df_production,
'BOM': df_BOM,
'WH-FG Storage' : df_whfg,
'Primary Freight' : df_pri_frt,
'Secondary Freight': df_sec_frt,
'Factory Operations': df_Factory_Operations,
'Calendar': df_Calendar,
'RM Supply': df_RM_Supply,
'LabourData': df_LabourData,
'Demand': df_demand,
'RM Transfer Restrictions':df_RM_transfer_res,
'Minimum Contract Labour Hours':df_Min_contract_labour_hours,
'RM Transfer':df_rmtransfer,
'Preparation Lines FG':df_prep_line_cap1,
'Preparation Lines Capacity' :df_prep_line_cap2,
    'TC_04_INV_06_FGCSInvNorm':TC_04_INV_06_FGCSInvNorm,
    'TC_04_INV_03_FGInv':TC_04_INV_03_FGInv,
    'TC_06_NegativeList_01_FGSupply':TC_06_NegativeList_01_FGSupply,
    'TC_06_NegativeList_02_RMSupply':TC_06_NegativeList_02_RMSupply,
    'TM_04_FIN_01_ICC': TM_04_FIN_01_ICC,
    'TM_03_CAL_01_Months': TM_03_CAL_01_Months
    
   }

# Specify the path to save your Excel file
file_path = combined_output_file_name


# Using ExcelWriter to write each DataFrame to a separate sheet
with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
    for sheet_name, dataframe in dataframes.items():
        dataframe.to_excel(writer, sheet_name=sheet_name, index=False)

    # Load the workbook and apply formatting
    workbook = writer.book
    for sheet_name in dataframes.keys():
        worksheet = workbook[sheet_name]
        worksheet.sheet_view.showGridLines = False  # Remove gridlines

        # Add one row and column at the top
        worksheet.insert_rows(1)
        worksheet.insert_cols(1)
        worksheet.cell(row=1, column=1)

        # Add table borders
        border = Border(left=Side(style='thin'), 
                        right=Side(style='thin'), 
                        top=Side(style='thin'), 
                        bottom=Side(style='thin'))
        for row in worksheet.iter_rows(min_row=2, min_col=2, max_col=worksheet.max_column):
            for cell in row:
                cell.border = border

        
# Set light blue color for headers
        for cell in worksheet.iter_cols(min_row=2, min_col=2):
            cell[0].fill = PatternFill(start_color="ADD8E6", end_color="ADD8E6", fill_type="solid")  # Light blue
    
        # Remove borders for the extra added row and column at the top
        for cell in worksheet['A1:C1'][0]:
            cell.border = None
# Autofit columns
        for col in worksheet.columns:
            max_length = 0
            column = col[0].column_letter  # Get the column name
            for cell in col:
                try:  # Necessary to avoid error on empty cells
                    if len(str(cell.value)) > max_length:
                        max_length = len(cell.value)
                except:
                    pass
            adjusted_width = (max_length + 2) * 1.2
            worksheet.column_dimensions[column].width = adjusted_width
 # Convert pixels to character units
        

        # Apply auto filters starting from the 2nd row in the B column
        #worksheet.auto_filter.ref = worksheet.dimensions
        last_row = worksheet.max_row
        last_column = worksheet.max_column
        range_str = f"B2:{get_column_letter(last_column)}{last_row}"
        worksheet.auto_filter.ref = range_str
# Save the Excel file
workbook.save(file_path)

print("All DataFrames are written to {}".format(file_path))
for table_name in dataframes.keys():
    print(table_name)

All DataFrames are written to 09-05-2024 Full Data_v0.9.xlsx
Country
Warehouse
Factory
Lines
FG SKUs
FG- Fact Trimming
RM SKUs
TrimmingSKUs
FG - M1OS
RM- M1OS
Production
BOM
WH-FG Storage
Primary Freight
Secondary Freight
Factory Operations
Calendar
RM Supply
LabourData
Demand
RM Transfer Restrictions
Minimum Contract Labour Hours
RM Transfer
Preparation Lines FG
Preparation Lines Capacity
TC_04_INV_06_FGCSInvNorm
TC_04_INV_03_FGInv
TC_06_NegativeList_01_FGSupply
TC_06_NegativeList_02_RMSupply
TM_04_FIN_01_ICC
TM_03_CAL_01_Months
